# ACCESS-NRI 2025 -- CMWG Workshop 
# Introduction to pyISSM

`pyISSM` is the Python API for the Ice-sheet and Sea-level System Model (ISSM) under development by ACCESS-NRI. This notebook provides an intial introduction to using pyISSM (v0.1) to load and navigate ISSM models.

`pyISSM` contains 5 primary sub-modules:
- `pyissm.plot`: Visualisation tools to support plotting of ISSM model components.
- `pyissm.param`: All model sub-classes used to parameterise ISSM models.
- `pyissm.data`: Tools for accessing and integrated common cryosphere-related datasets onto ISSM models.
- `pyissm.model`: Tools for interacting with ISSM models, including model IO.
- `pyissm.learn`: Tutorials and examples of working with ISSM models.

<div class="alert alert-block alert-info">
<b>AIM:</b> This notebook focuses on exploring existing ISSM models using pyISSM to ensure users are familiar with ISSM model structures before continuing onto model visualisation.
</div>

In [1]:
# Import required modules
import numpy as np

# Import pyISSM
# NOTE: Here, we assume that the `pyISSM` git repository has been cloned onto your machine. If `pyISSM` has been installed using `conda`, skip the `sys` calls.
import sys
import socket
hostname = socket.gethostname()
if 'gadi' in hostname:
    sys.path.append('/g/data/nf33/access-nri/pyISSM/src/')
else:
    sys.path.append('/Users/lawrence.bird/pyISSM/src/')
import pyissm

# 1. pyISSM `Model()` Class

`pyISSM` is developed around a central `Model()` class. Each Model contains a series of sub-classes that define model components and parameters (e.g. `mesh`, `geometry`). Each Model and sub-class contains detailed information that can be interrogated/explored interractively. 

Below, let's load an existing model and explore some of the sub-classes.

In [2]:
# Load an existing ISSM model onto md

if 'gadi' in hostname:
    # md = pyissm.model.io.load_model('/g/data/nf33/access-nri/CMWG_workshop_2025/sample_models/Greenland.HistoricTransient_200yr.nc')
    md = pyissm.model.io.load_model('/g/data/nf33/access-nri/CMWG_workshop_2025/sample_models/zrun_yearly_aSMB_HadGEM2_ctrl_2300_again.nc')
else:
    md = pyissm.model.io.load_model('/Users/lawrence.bird/access-dev/sampleModels/Greenland.HistoricTransient_200yr.nc')

ℹ️ Legacy classtype 'mesh2d.mesh2d' mapped to 'mesh.mesh2d'
ℹ️ Legacy classtype 'SMBforcing.SMBforcing' mapped to 'smb.default'
⚠️️ classtype does not exist for group basalforcings. Skipping...
ℹ️ Legacy classtype 'matice.matice' mapped to 'materials.ice'
ℹ️ Legacy classtype 'friction.friction' mapped to 'friction.default'
ℹ️ Legacy classtype 'timestepping.timestepping' mapped to 'timestepping.default'
ℹ️ Legacy classtype 'dsl.dsl' mapped to 'dsl.default'
ℹ️ Legacy classtype 'solidearth.solidearth' mapped to 'solidearth.earth'
⚠️ Unknown classtype verbose.verbose. Skipping...
⚠️ Unknown classtype toolkits.toolkits. Skipping...
⚠️ Unknown classtype gadi.gadi. Skipping...
ℹ️ Legacy classtype 'hydrologyshreve.hydrologyshreve' mapped to 'hydrology.shreve'
ℹ️ Legacy classtype 'calving.calving' mapped to 'calving.default'
ℹ️ Legacy classtype 'frontalforcings.frontalforcings' mapped to 'frontalforcings.default'
ℹ️ Legacy classtype 'love.love' mapped to 'love.default'
ℹ️ Legacy classtype 'm1qn

<div class="alert alert-block alert-warning">
<b>Backwards-compatability:</b> A series of informative warnings are provided. These highlight "re-mapping" that is completed to ensure backwards-compatibility with ISSM models developed using legacy Python infrastructure and in MATLAB. Any sub-classes that are not yet implemented in pyISSM are highlighted (e.g. verbose, toolkits, generic). Any classes that are missing in the model file are highlighted (e.g. basalforcings).
</div>

In [3]:
# Simply examining the md object reveals the sub-classes included in all ISSM models.

md

   ISSM Model Class                         
                                            
               mesh:  mesh properties         
               mask:  defines grounded and gloating elements 
           geometry:  surface elevation, bedrock topography, ice thickness, ... 
          constants:  physical constants      
                smb:  surface mass balance    
      basalforcings:  bed forcings            
          materials:  material properties     
             damage:  damage propagation laws 
           friction:  basal friction / drag properties 
       flowequation:  flow equations          
       timestepping:  timestepping for transient models 
     initialization:  initial guess / state   
              rifts:  rifts properties        
         solidearth:  solidearth inputs and settings 
                dsl:  dynamic sea level       
              debug:  debugging tools (valgrind, gprof 
            verbose:  verbosity level in solve 
           settings:  sett

# 2. `pyISSM` Model sub-classes

Above, we can see that the ISSM `Model()` class comprises 43 model sub-classes. Each sub-class contains specific model parameters and settings that control the model behaviour. For example, `md.mesh` contains all of the mesh parameters and `md.geometry` contains all of the initial geometry fields (e.g. bed topography, surface elevation, ice thickness etc.).

Parameters within each model subclass are assigned/accessed using standard `.` notation. For example, bed topography can be assigned/accessed as `md.geometry.bed` and the ice thickness can be assigned/accessed as `md.geometry.thickness`.

Most data fields contained in model sub-classes are `numpy.ndarray` classes under-the-hood. This ensures they're easily accessible and can be manipulated with other popular Python modules.

Let's explore a few different model sub-classes in the cells below.

## 2.1 `md.mesh`

Let's review the `md.mesh` sub-class. Accessing the `md.mesh` sub-class shows the parameters/data fields available and their associated shape/size or scalar value.

In [4]:
# Access the model mesh
md.mesh

   2D tria Mesh (horizontal):
      Elements and vertices:
         numberofelements       : 199226          -- number of elements
         numberofvertices       : 99870           -- number of vertices
         elements               : (199226, 3)     -- vertex indices of the mesh elements
         x                      : (99870,)        -- vertices x coordinate [m]
         y                      : (99870,)        -- vertices y coordinate [m]
         edges                  : (299095, 4)     -- edges of the 2d mesh (vertex1 vertex2 element1 element2)
         numberofedges          : 299095          -- number of edges of the 2d mesh

      Properties:
         vertexonboundary       : (99870,)        -- vertices on the boundary of the domain flag list
         segments               : (512, 3)        -- edges on domain boundary (vertex1 vertex2 element)
         segmentmarkers         : (512,)          -- number associated to each segment
         vertexconnectivity     : N/A       

Within ISSM, data fields are typically defined on model vertices, or model elements. Above, we see that this model has 199,226 elements and 99,870 elements. The vertices are defined by the X and Y coordinates.

We can extract the X/Y coordinates as numpy arrays:

In [5]:
# Extract the X and Y coordinates from the model mesh
x = md.mesh.x
y = md.mesh.y

# The extracted coordinates are simply numpy arrays
print(f'Model x-coordinates [m]: {x}\n')
print(f'Model y-coordinates [m]: {y}')

Model x-coordinates [m]: [-2454342.45605584 -2336563.52347844 -2229491.76658989 ...
  1136689.51706827  1381652.71625784 -2576497.06419619]

Model y-coordinates [m]: [ 1729886.94743119  1612108.01485379  1467561.14305425 ...
 -2061844.23304862  1713053.24915904  1706247.23560796]


## 2.2 `md.geometry`

Let's review the `md.geometrty` sub-class. Accessing the `md.geometry` sub-class shows the parameters/data fields available and their associated shape/size.

In [6]:
md.geometry

   geometry parameters:
         surface                : (99870,)        -- ice upper surface elevation [m]
         thickness              : (99870,)        -- ice thickness [m]
         base                   : (99870,)        -- ice base elevation [m]
         bed                    : (99870,)        -- bed elevation [m]
         hydrostatic_ratio      : N/A             -- hydrostatic ratio for floating ice

Here, we can see that `md.geometry.surface`, `md.geometry.thickness`, and `md.geometry.base` are defined on vertices, as the shape of the fields is consistent with the number of vertices (`md.mesh.numberofvertices`) in the model (which is also consistent with the size of X and Y coordinates we extracted above).

In [7]:
print(f'The model has {md.mesh.numberofvertices} vertices and {md.mesh.numberofelements} elements.')
print(f'With {md.geometry.surface.shape[0]} values, the ice upper surface elevation is defined on model vertices')

The model has 99870 vertices and 199226 elements.
With 99870 values, the ice upper surface elevation is defined on model vertices


Because data fields are stored as a `numpy.ndarray`, they can easily be interrogated. For example, we can calculate the mean surface elevation and range of ice thickness, like so:

In [8]:
print(f'Mean ice surface elevation: {np.round(md.geometry.surface.mean(), 2)}')
print(f'Range of ice thickness: {np.round(md.geometry.thickness.min(), 2)} to {np.round(md.geometry.thickness.max(), 2)}.')

Mean ice surface elevation: 877.06
Range of ice thickness: 1.0 to 4395.72.
